## Ensemble LDA

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', -1)
path = "/content/gdrive/My Drive/data/gensim"
train = pd.read_csv(path + "/ag_news_train.csv")

<ipython-input-5-ff92c4043bad>:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [ ]:
from gensim.parsing.preprocessing import preprocess_string
text_tokenized = []
for doc in train['Description']:
   k = preprocess_string(doc)
   text_tokenized.append(k)
text_tokenized[0:3]


[['reuter',
  'short',
  'seller',
  'wall',
  'street',
  'dwindl',
  'band',
  'ultra',
  'cynic',
  'see',
  'green'],
 ['reuter',
  'privat',
  'invest',
  'firm',
  'carlyl',
  'group',
  'reput',
  'make',
  'time',
  'occasion',
  'controversi',
  'plai',
  'defens',
  'industri',
  'quietli',
  'place',
  'bet',
  'market'],
 ['reuter',
  'soar',
  'crude',
  'price',
  'plu',
  'worri',
  'economi',
  'outlook',
  'earn',
  'expect',
  'hang',
  'stock',
  'market',
  'week',
  'depth',
  'summer',
  'doldrum']]

In [ ]:
from gensim.corpora import Dictionary
gensim_dictionary = Dictionary()
bow_corpus = [gensim_dictionary.doc2bow(doc, allow_update=True) for doc in text_tokenized]
print(bow_corpus[:3])
id_words = [[(gensim_dictionary[id], count) for id, count in line] for line in bow_corpus]
print(id_words)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1)], [(4, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1)], [(4, 1), (20, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1)]]


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
from gensim.models import TfidfModel
tfidf = TfidfModel(bow_corpus) #, smartirs=’npu’)
tfidf_corpus = tfidf[bow_corpus]
print(tfidf_corpus[:3])
id_words = [[(gensim_dictionary[id], count) for id, count in line] for line in bow_corpus]
print(id_words)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



### Save the file

In [ ]:
from gensim.test.utils import datapath
dict_file = datapath(path + "/gensim_dictionary_AGnews")
gensim_dictionary.save(dict_file)


In [ ]:
from gensim import corpora
# Save in the matrix market format
corpora.MmCorpus.serialize(path + "/BoW_AGnews_corpus.mm", bow_corpus)
# Load
bow_corpus = corpora.MmCorpus(path + "/BoW_AGnews_corpus.mm")
bow_corpus[0]


[(0, 1.0),
 (1, 1.0),
 (2, 1.0),
 (3, 1.0),
 (4, 1.0),
 (5, 1.0),
 (6, 1.0),
 (7, 1.0),
 (8, 1.0),
 (9, 1.0),
 (10, 1.0)]

In [ ]:
import pickle
file = open(path + "/BoW_AGnews_corpus.pkl", 'wb')
pickle.dump(bow_corpus, file)
file.close()
# open a file, where you stored the pickled data
file = open(path + "/BoW_AGnews_corpus.pkl", 'rb')
bow_corpus = pickle.load(file)
# close the file
file.close()
bow_corpus[0]


[(0, 1.0),
 (1, 1.0),
 (2, 1.0),
 (3, 1.0),
 (4, 1.0),
 (5, 1.0),
 (6, 1.0),
 (7, 1.0),
 (8, 1.0),
 (9, 1.0),
 (10, 1.0)]

### Build the Ensemble model

In [13]:
from gensim.corpora.dictionary import Dictionary
from gensim.models import ensemblelda
elda_bow = ensemblelda.EnsembleLda(corpus=bow_corpus, id2word=gensim_dictionary, num_topics=100)
elda_tfidf = ensemblelda.EnsembleLda(corpus=tfidf_corpus, id2word=gensim_dictionary, num_topics=100)


In [14]:
from gensim.test.utils import datapath
# Save the eLDA model trained on BOW data
elda_bow_file = datapath(path + "/eLDA_bow_AGnews")
elda_bow.save(elda_bow_file)
# Save the eLDA model trained on TF-IDF data
elda_tfidf_file = datapath(path + "/eLDA_tfidf_AGnews")
elda_tfidf.save(elda_tfidf_file)


In [15]:
import pprint as pp
pp.pprint(elda_bow.print_topics())
len(elda_bow.print_topics())

[(0,
  '0.092*"reuter" + 0.085*"fullquot" + 0.076*"stock" + 0.068*"target" + '
  '0.068*"com" + 0.061*"investor" + 0.055*"ticker" + 0.055*"http" + '
  '0.052*"www" + 0.051*"href"'),
 (1,
  '0.366*"quot" + 0.070*"research" + 0.039*"said" + 0.029*"profil" + '
  '0.016*"decid" + 0.014*"new" + 0.012*"winter" + 0.012*"sai" + 0.009*"presid" '
  '+ 0.009*"carri"')]


2

In [16]:
# Create a new corpus, made of previously unseen documents.
other_texts = [
    ['champion', 'hockei', 'qualifi'],
    ['survey', 'tournament', 'world']
]

other_corpus = [gensim_dictionary.doc2bow(text) for text in other_texts]
unseen_doc = other_corpus[0]
vector = elda_bow[unseen_doc]  # get topic probability distribution for a document
vector

[(0, 0.25289607), (1, 0.7471039)]